In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from langchain_huggingface import HuggingFacePipeline
import torch

/home/gavinjwl/my-baymax/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model_id = "microsoft/Phi-3-mini-4k-instruct"
tokenizer = AutoTokenizer.from_pretrained(model_id)

In [3]:
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    attn_implementation="flash_attention_2",
)

Loading checkpoint shards: 100%|██████████| 2/2 [00:06<00:00,  3.09s/it]


In [11]:
pipe = pipeline(
    task="text-generation",
    # task="question-answering",
    model=model, tokenizer=tokenizer,
    max_new_tokens=100,
    top_k=1,
    temperature=0.1,
    do_sample=True,
)


In [12]:
llm = HuggingFacePipeline(pipeline=pipe)
llm.invoke("""
<|system|>
You are a helpful assistant and only anwser the question.<|end|>
<|user|>
where is Taiwan?<|end|>
<|assistant|> 
""")

'\n<|system|>\nYou are a helpful assistant and only anwser the question.<|end|>\n<|user|>\nwhere is Taiwan?<|end|>\n<|assistant|> \n Taiwan is located in East Asia, situated off the southeastern coast of China. It is an island nation that lies approximately 180 kilometers (110 miles) off the southeastern coast of China, across the Taiwan Strait. The country comprises both the island of Taiwan and several smaller islands, including Penghu, Kinmen, and Matsu. Taiwan is known for its beautiful landscapes, rich history, and vibrant culture.'

In [15]:
from langchain_core.prompts import PromptTemplate

template = """
<|system|>
You are a helpful assistant and only anwser the question.<|end|>
<|user|>
{question}<|end|>
<|assistant|>
"""

prompt = PromptTemplate.from_template(template)
chain = prompt | llm
print(chain.invoke({
    "question": "what is steam?"
}))


<|system|>
You are a helpful assistant and only anwser the question.<|end|>
<|user|>
what is steam?<|end|>
<|assistant|>
 Steam is the gaseous phase of water, which occurs when water boils and transitions from its liquid state to a gas. This process happens when water reaches its boiling point, which is 100 degrees Celsius (212 degrees Fahrenheit) at standard atmospheric pressure. Steam is invisible, but it can be seen when it condenses into tiny water droplets, forming clouds or mist. Steam is used in various applications, including
